In [1]:
import numpy as np 
import pandas as pd 

In [2]:
df  = pd.read_csv('BostonHousing.csv')
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,NaN,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [3]:
df.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'MEDV'],
      dtype='object')

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X = df.drop('MEDV', axis=1)   # Features
y = df['MEDV']  #Target

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [7]:
from xgboost import XGBRegressor
xgb = XGBRegressor()

In [8]:
xgb.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [9]:
from sklearn.metrics import r2_score,mean_squared_error

In [10]:
y_pred = xgb.predict(X_test)
y_pred_train= xgb.predict(X_train)

In [11]:
print('r2 score',r2_score(y_test,y_pred))
print('r2 score',r2_score(y_train,y_pred_train))

r2 score 0.9287327392666798
r2 score 0.9999989877975602


In [12]:
print('mse:',mean_squared_error(y_test,y_pred))
print('mse:',mean_squared_error(y_train,y_pred_train))

mse: 5.2262978450475055
mse: 8.793347130753561e-05


In [13]:
from sklearn.model_selection import GridSearchCV

In [14]:
param_grid = {
    'estimators':[100,50,200],  #number of argument
    'max_depth':[3,6,9],
    'eta':[0.01,0.1,0.3],  #Learning Rate
    'subsample':[0.5,0.75,1.0]
}

In [15]:
xgb_reg = XGBRegressor(objective='reg:squarederror',verbosity=0)

In [16]:
grid_search = GridSearchCV(estimator=xgb_reg,param_grid = param_grid, scoring =   'neg_mean_squared_error',cv=5,n_jobs=-1,verbose=2)

In [17]:
grid_search.fit(X_train,y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'estimators': [100, 50, 200], 'eta': [0.01, 0.1, 0.3],
                         'max_depth': [3, 6, 9],
                         'subsample': [0.5, 0.75, 1.0]},
             scoring='neg_mean_squared_error', verbose=2)

In [18]:
best_params = grid_search.best_params_
best_score = grid_search.best_score_

In [19]:
print('Best Parameter:',best_params)
print('Best Score:',best_score)

Best Parameter: {'estimators': 100, 'eta': 0.3, 'max_depth': 3, 'subsample': 0.75}
Best Score: -13.201952678881266
